In [1]:
import os
import random
from sklearn.model_selection import train_test_split
import torchvision.transforms.functional as TF

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
from tqdm.auto import tqdm

import numpy as np
import matplotlib.pyplot as plt

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [3]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    device = torch.device("mps")

In [4]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3
width_out = 128
height_out = 128

In [5]:
rt_path = "datasets/"
TRAIN_PATH = os.path.join(rt_path, "stage1_train/")
TEST_PATH = os.path.join(rt_path, "stage1_test/")

train_dir = os.listdir(TRAIN_PATH)
test_dir = os.listdir(TEST_PATH)
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

x_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
y_train = np.zeros((len(train_ids), height_out, width_out, 1), dtype=bool)
x_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)

In [6]:
for i, id_ in enumerate(train_ids):
    path = TRAIN_PATH + id_
    img = cv2.imread(path+'/images/'+id_+'.png')
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    x_train[i] = img

    mask = np.zeros((height_out, width_out, 1), dtype=bool)
    mask_path = path+'/masks/'
    for id_ in os.listdir(mask_path):
        mask_ = cv2.imread(mask_path+id_, 0)
        mask_ = cv2.resize(mask_, (height_out, width_out))
        mask_ = np.expand_dims(mask_, axis=-1)
        mask = np.maximum(mask, mask_)
    y_train[i] = mask

In [7]:
np.save("datasets/x_train.npy", x_train)
np.save("datasets/y_train.npy", y_train)

In [8]:
class CustomDataset(Dataset):
    def __init__(self, images_np, masks_np):
        self.images_np = images_np
        self.masks_np = masks_np

    def transform(self, image_np, mask_np):
        ToPILImage = transforms.ToPILImage()
        image = ToPILImage(image_np)
        mask = ToPILImage(mask_np.astype(np.int32))

        image = TF.to_tensor(image)
        mask = TF.to_tensor(mask)
        return image, mask

    def __len__(self):
        return len(self.images_np)

    def __getitem__(self, idx):
        image_np = self.images_np[idx]
        mask_np = self.masks_np[idx]
        image, mask = self.transform(image_np, mask_np)

        return image, mask

In [9]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=0)
train_dataset = CustomDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_dataset = CustomDataset(x_val, y_val)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

In [10]:
from nets.minseongNet.ResWNet import ResWNet
from nets.minseongNet.ResUNet_F2R import ResUNet_F2R, Dacon_ResUNet

# model = Dacon_ResUNet(num_classes=1).to(device)
model = ResWNet().to(device)

/Users/kweonminseong/.pyenv/versions/ResWNet/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/kweonminseong/.pyenv/versions/ResWNet/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/kweonminseong/.pyenv/versions/ResWNet/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights

In [11]:
epochs=25 # 3 -> 25
alpha=5
batch_size = 16 # 각자 메모리 상태에 맞게 변경하셔도 됩니다.
# nn.CrossEntropyLoss() (paper) -> BCELoss()
criterion=nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
train_iou_sum = 0
valid_iou_sum = 0

In [12]:
def iou(pred, target, n_classes = 2):

    iou = []
    pred = pred.view(-1)
    target = target.view(-1)

    # Ignore IoU for background class ("0")
    for cls in range(1, n_classes):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = (pred_inds[target_inds]).long().sum().data.cpu().item()
        union = pred_inds.long().sum().data.cpu().item() + target_inds.long().sum().data.cpu().item() - intersection

        if union == 0:
            iou.append(float('nan'))  # If there is no ground truth, do not include in evaluation
        else:
            iou.append(float(intersection) / float(max(union, 1)))

    return sum(iou)

In [13]:
def iou_metric(y_pred, y_true, n_classes = 2):
    miou = []
    for i in np.arange(0.5, 1.0, 0.05):
        y_pred_ = (y_pred > i)
        iou_init = iou(y_pred_, y_true, n_classes = n_classes)
        miou.append(iou_init)

    return sum(miou)/len(miou)

In [14]:
train_loss_list = []
val_loss_list = []
train_iou_list = []
val_iou_list = []

for epoch in range(epochs):
    model.train()
    train_loss = 0
    train_iou = 0

    for image, mask in tqdm(train_loader):
        image = image.to(device)
        mask = mask.to(device)
        optimizer.zero_grad()
        outputs = model(image.float())
        print(outputs.shape)
        loss = criterion(outputs.float(), mask.float())
        train_loss += loss

        train_iou += iou_metric(outputs, mask)
        rev_iou = 16 - iou_metric(outputs, mask)
        loss += alpha * rev_iou

        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        valid_loss = 0
        valid_iou = 0

        for image_val, mask_val in valid_loader:
            image_val = image_val.to(device)
            mask_val = mask_val.to(device)
            output_val = model(image_val.float())
            valid_loss += criterion(output_val.float(), mask_val.float())
            valid_iou += iou_metric(output_val, mask_val)

    print("Epoch ", epoch + 1, " Training Loss: ", train_loss/len(train_loader), "Validation Loss: ", valid_loss/len(valid_loader))
    print("Training IoU: ", train_iou/len(train_loader), "Validation IoU: ", valid_iou/len(valid_loader))
    train_iou_sum += train_iou/len(train_loader)
    valid_iou_sum += valid_iou/len(valid_loader)

    # visualization
    train_loss_list.append(train_loss/len(train_loader))
    val_loss_list.append(valid_loss/len(valid_loader))
    train_iou_list.append(train_iou/len(train_loader))
    val_iou_list.append(valid_iou/len(valid_loader)
                        )

print("Training Mean IoU: {:.2f}".format(train_iou_sum/epochs), " Validation Mean IoU: {:.2f}".format(valid_iou_sum/epochs))

  0%|          | 0/32 [00:00<?, ?it/s]

/Users/kweonminseong/.pyenv/versions/ResWNet/lib/python3.9/site-packages/torch/nn/functional.py:3737: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


torch.Size([16, 1, 128, 128])


/var/folders/26/tc97y9y91mqdvfbhjdvdfcsm0000gn/T/ipykernel_6467/487936267.py:11: UserWarning: MPS: no support for int64 reduction ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:144.)
  intersection = (pred_inds[target_inds]).long().sum().data.cpu().item()


torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([6, 1, 128, 128])
Epoch  1  Training Loss:  tensor(0.3393, device='mps:0', grad_fn=<DivBa

...

torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([6, 1, 128, 128])
Epoch  23  Training Loss:  tensor(0.0738,

  0%|          | 0/32 [00:00<?, ?it/s]

torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([6, 1, 128, 128])
Epoch  24  Training Loss:  tensor(0.0682,

  0%|          | 0/32 [00:00<?, ?it/s]

torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 128, 128])
torch.Size([6, 1, 128, 128])
Epoch  25  Training Loss:  tensor(0.0680,